### Big Query Guide

#### Connect to Big Query

In [1]:
import pandas as pd
from google.cloud import bigquery
client = bigquery.Client(project="project_name")

In [ ]:
final_df=pd.read_csv('result.csv',index_col=0)
final_df.head()

,Dashboard ID,Dashboard Title,is_Deleted,Created At,Updated At,Last Updater Name,Last Viewed At,Element Title,Column Name,Column Label,Column Label_df2,Filter title
0,7,AR Aging Report,False,2022-07-29 15:38:28+00:00,2022-08-16 20:55:47+00:00,Internal User,2024-11-08 22:24:39+00:00,NaN,v_ar_aging_tiers.csm_name,Csm Name,CSM Name,NaN
1,7,AR Aging Report,False,2022-07-29 15:38:28+00:00,2022-08-16 20:55:47+00:00,Internal User,2024-11-08 22:24:39+00:00,NaN,v_ar_aging_tiers.customer_id,Customer ID,NaN,NaN
2,7,AR Aging Report,False,2022-07-29 15:38:28+00:00,2022-08-16 20:55:47+00:00,Internal User,2024-11-08 22:24:39+00:00,NaN,v_ar_aging_tiers.customer_name,Customer Name,NaN,NaN
3,7,AR Aging Report,False,2022-07-29 15:38:28+00:00,2022-08-16 20:55:47+00:00,Internal User,2024-11-08 22:24:39+00:00,NaN,v_ar_aging_tiers.less_than_5,Less Than 5,< 5 days late,NaN
4,7,AR Aging Report,False,2022-07-29 15:38:28+00:00,2022-08-16 20:55:47+00:00,Internal User,2024-11-08 22:24:39+00:00,NaN,v_ar_aging_tiers.five_to_30,Five to 30,5 to 30 days late,NaN


#### Read Table `bigquery -> pandas`

In [ ]:
query = f"SELECT * FROM dataset_2.v_view limit 5"
dataframe = client.query(query).to_dataframe()
dataframe

#### Write Table `pandas -> bigquery`


In [ ]:
table_id = 'project_name.dataset_2.looker_report'

job_config = bigquery.LoadJobConfig(
    # write_disposition=bigquery.WriteDisposition.WRITE_APPEND # append (default if no config is set)
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE # overwrite
)

# Load the DataFrame and overwrite the table
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()

LoadJob<project=project_name, location=US, id=b85e9361-b55b-499b-875c-72a9dc33d22d>

#### Copy Table `bigquery -> bigquery`

In [ ]:
source_table_id = "project.dataset.source_table"
destination_table_id = "project.dataset.destination_table"
# job = client.copy_table(source_table_id, destination_table_id)
job.result()
print("Table copied.")


#### Delete Table

In [2]:
table_id = 'dataset_1.election_health_benefit_election'
# client.delete_table(table_id, not_found_ok=True)
print(f"Table {table_id} deleted successfully.")

Table dataset_1.election_health_benefit_election deleted successfully.


#### Get Table Schema

In [ ]:
table = client.get_table("project.dataset.table")
for field in table.schema:
    print(field.name, field.field_type)


#### Get View Query

In [ ]:
# Define the view ID
view_id = "project_name.dataset_2.v_view"  # Replace with your view ID

# Get the view details
view = client.get_table(view_id)  # API call

# Print the view's query
if view.view_query:
    print("View Query:")
    print(view.view_query)
else:
    print("The specified table is not a view.")


View Query:
with bcos2 as (
	select 
	\...)
order by company_invite_date desc, company_name


### Copy `dataset_0` tables to `dataset_1`

In [16]:
source_dataset_id = 'dataset_0'
target_dataset_id = 'dataset_1'

# List tables in the source dataset
tables = client.list_tables(target_dataset_id)
tables

In [8]:
from google.cloud.exceptions import NotFound
import time
# Specify the source and target datasets
# source_dataset_id = 'dataset_0'
# target_dataset_id = 'dataset_02'

source_dataset_id = 'project_name.dataset_0'
target_dataset_id = 'project_name.dataset_1'

print(f'Source: {source_dataset_id}\nTarget:{target_dataset_id}')
time.sleep(5)

# List tables in the source dataset
tables = client.list_tables(source_dataset_id)
old_tables=[]

for table in tables:
    source_table_id = f"{source_dataset_id}.{table.table_id}"
    target_table_id = f"{target_dataset_id}.{table.table_id}"
    
    # Check if the table already exists in the target dataset
    try:
        client.get_table(target_table_id)
        old_tables.append(target_table_id)
        # print(f"Table {target_table_id} already exists. Skipping creation.")
        continue
    except NotFound:
        # Get the table schema
        source_table = client.get_table(source_table_id)
        table = bigquery.Table(target_table_id, schema=source_table.schema)
        client.create_table(table) # uncomment this
        print(f"{target_table_id} Created!")
    
    # Create a new table with the same schema in the target dataset
    
    

print('All Good! 👍')

Source: project_name.dataset_0
Target:project_name.dataset_1
project_name.dataset_1.table_0 Created!
project_name.dataset_1.table_1 Created!
project_name.dataset_1.table_2 Created!
project_name.dataset_1.table_3 Created!
project_name.dataset_1.table_4 Created!
project_name.dataset_1.table_5 Created!
project_name.dataset_1.table_6 Created!
All Good! 👍


In [20]:
# takes 86 seconds 
source_dataset_id = 'project_name.dataset_0'
# target_dataset_id = 'project_name.dataset_02'

# List tables in the source dataset
tables = client.list_tables(source_dataset_id)

for table in tables:
        table_id = f"{source_dataset_id}.{table.table_id}"
        
        # Construct the DELETE query
        query = f"DELETE FROM `{table_id}` WHERE TRUE"
        print(query)
        
        # Run the query to delete all data
        # query_job = client.query(query)
        # query_job.result()